In [1]:
import datetime, requests, time
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# api calls
apiBase = 'https://www.boardgamegeek.com/xmlapi2/'
gameInfo = 'thing?id='
userInfo = 'user?name='
playsInfoByUser = 'plays?username='
playsInfoByGame = 'plays?id='

In [44]:
# retrieve API call
def get_game_metadata(gameIdList):
    """supply a list of game id's you want to retrive game metadata on.  Output is XML."""
    stringify = ','.join(gameIdList)
    response = requests.get(f'{apiBase}{gameInfo}{stringify}').content
    # add a short sleep so API doesn't throttle - we aren't making that makny calls
    time.sleep(5)
    return response

In [45]:
def listify_game_metadata(apiResponse):
    """The parses the game metadata XML into a list of lists for import"""
    gameMetadata = []
    gameCategories = []
    gameMechanisms = []
    importDate = datetime.datetime.strftime(datetime.datetime.now().date(), '%Y-%m-%d')

    # game info
    tree = ET.fromstring(apiResponse)

    # iterate through response top level 'item'
    for item in tree.findall('item'):

        # define metadata we want to collect since Type & Id are attributes at the top level we will set them here
        itemType = item.attrib['type']
        itemId = item.attrib['id']      
        itemName = None                 
        itemYearPublished = None                
        itemMinPlayers = None           
        itemMaxPlayers = None           
        itemPlayTime = None             
        itemCategories = []
        itemMechanisms = []
        itemThumbnail = None
        
        # get itemName
        names = item.findall('name')
        for name in names:
            if name.attrib['type'].lower() == 'primary':
                name = name.attrib['value']
                itemName = name
        
        # get itemYearPublished
        publishedYear = item.find('yearpublished')
        itemYearPublished = publishedYear.attrib['value']

        # get itemMinPlayers
        minPlayers = item.find('minplayers')
        itemMinPlayers = minPlayers.attrib['value']

        # get itemMaxPlayers
        maxPlayers = item.find('maxplayers')
        itemMaxPlayers = maxPlayers.attrib['value']

        # get itemPlayingTime
        playingTime = item.find('playingtime')
        itemPlayTime = playingTime.attrib['value']

        # get itemCategories
        categories = item.findall('link')
        for category in categories:
            if 'categor' in category.attrib['type'].lower():
                itemCategories.append(category.attrib['value'])

        # get itemCategories
        mechanisms = item.findall('link')
        for mechanism in mechanisms:
            if 'mechanic' in mechanism.attrib['type'].lower():
                itemMechanisms.append(mechanism.attrib['value'])

        # get itemThumbnail
        thumbnail = item.find('thumbnail')
        itemThumbnail = thumbnail.text

        gameMetadata += tuple([(itemId, itemType, itemName, itemYearPublished, None, itemMinPlayers, itemMaxPlayers,
                         itemPlayTime, itemThumbnail, importDate)])
        
        gameCategories += tuple([(itemId, x) for x in itemCategories])

        gameMechanisms += tuple([(itemId, x) for x in itemMechanisms])
        
    #return [gameMetadata]
    return [gameCategories]
    #return [gameMechanisms]

In [46]:

idList= ['174430', '161936'] #'224517']
categorylist=[]

for a in idList:
    response = get_game_metadata(str(a))
    res=listify_game_metadata(response)
    count=0
    for i in res:
        categorylist.append([a,res[count][1]])
        count+=1
        
    

    
categorylist



[['174430', ('1', 'Negotiation')], ['161936', ('1', 'Negotiation')]]

In [48]:
idList= ['174430', '161936']
categorylist=[]
for a in idList:
    response = get_game_metadata(str(a))
    res=listify_game_metadata(response)
    for t in res[0]:
        categorylist.append([a,t[1]])
print(res)
categorylist

[[('1', 'Economic'), ('1', 'Negotiation'), ('1', 'Political'), ('6', 'Civilization'), ('6', 'Nautical'), ('1', 'Economic'), ('1', 'Negotiation'), ('1', 'Political'), ('9', 'Exploration'), ('3', 'Abstract Strategy'), ('3', 'Medieval'), ('6', 'Civilization'), ('6', 'Nautical')]]


[['174430', 'Economic'],
 ['174430', 'Negotiation'],
 ['174430', 'Political'],
 ['174430', 'Abstract Strategy'],
 ['174430', 'Ancient'],
 ['174430', 'Ancient'],
 ['174430', 'Abstract Strategy'],
 ['174430', 'Medieval'],
 ['161936', 'Economic'],
 ['161936', 'Negotiation'],
 ['161936', 'Political'],
 ['161936', 'Civilization'],
 ['161936', 'Nautical'],
 ['161936', 'Economic'],
 ['161936', 'Negotiation'],
 ['161936', 'Political'],
 ['161936', 'Exploration'],
 ['161936', 'Abstract Strategy'],
 ['161936', 'Medieval'],
 ['161936', 'Civilization'],
 ['161936', 'Nautical']]

In [6]:
type(listify)

list

In [51]:
categories=pd.DataFrame(categorylist,columns=['1','2'])
categories.groupby(['1'])

In [ ]:
categories=pd.DataFrame(listify,columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])

In [ ]:
categories